# BigQuery Getting Started

### Installing bq console

```bash
$ echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
$ sudo apt-get install apt-transport-https ca-certificates gnupg
$ curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key --keyring /usr/share/keyrings/cloud.google.gpg add -
$ sudo apt-get update && sudo apt-get install google-cloud-sdk

$ gcloud init --console-only
Your current configuration has been set to: [default]

You must log in to continue. Would you like to log in (Y/n)?Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=2323

Enter verification code: 32kl;22l
You are logged in as: [johannes@fluquid.com].

Pick cloud project to use:
 [1] asdf-123233
 [6] Create a new project
Please enter numeric choice or text value (must exactly match list
item):  1

Your current project has been set to: [asdf-123233].

* Run `gcloud --help` to see the Cloud Platform services you can interact with. And run `gcloud help COMMAND` to get help on any gcloud command.
```

In [49]:
%%time
!bq query --use_legacy_sql=false 'SELECT sum(size_bytes)/pow(10,12) as size_tb FROM `gdelt-bq:gdeltv2.__TABLES__`;'

Waiting on bqjob_r70a7e05e807903c5_0000016eb157f833_1 ... (0s) Current status: DONE   
+-----------------+
|     size_tb     |
+-----------------+
| 79.526084032165 |
+-----------------+
CPU times: user 125 ms, sys: 109 ms, total: 234 ms
Wall time: 11.7 s


## Jupyter bigquery extension

* setup authentication: https://console.cloud.google.com/apis/credentials/serviceaccountkey
* `export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/[FILE_NAME].json"`

In [1]:
%load_ext google.cloud.bigquery
import google.cloud.bigquery.magics

# use bqstorage for large query results
google.cloud.bigquery.magics.context.use_bqstorage_api = True

In [3]:
%%bigquery
SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = 'TX'
    LIMIT 5

,name
0,Mary
1,Ruby
2,Annie
3,Willie
4,Ruth


## Pandas integration

In [34]:
import pandas

sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = 'TX'
    LIMIT 5
"""

pandas.read_gbq(sql, dialect='standard', use_bqstorage_api=True)

Downloading: 100%|██████████| 5/5 [00:01<00:00,  2.70rows/s]


,name
0,Mary
1,Ruby
2,Annie
3,Willie
4,Ruth


In [35]:
from google.cloud import bigquery

client = bigquery.Client()
client.query(sql).to_dataframe()

,name
0,Mary
1,Ruby
2,Annie
3,Willie
4,Ruth


### Inserting tables from Pandas

In [16]:
'''
table_id = 'my_dataset.new_table'
df.to_gbq(table_id)
'''
None

### ODBC/ JDBC

In [23]:
import pyodbc

if False:
    cnxn = pyodbc.connect('DRIVER={CData ODBC Driver for GoogleBigQuery};OAuthAccessToken=MyOAuthAccessToken;ProjectId=NameOfProject;DatasetId=NameOfDataset')
    cnxn = pyodbc.connect('DSN=CData GoogleBigQuery Source Sys;User=MyUser;Password=MyPassword')
    cursor = cnxn.cursor()
    cursor.execute("SELECT actor.attributes.email, repository.name FROM publicdata:samples.github_nested WHERE repository.name = 'EntityFramework'")
    rows = cursor.fetchall()
    for row in rows:
        print(row.actor.attributes.email, row.repository.name)

## Ibis Framework

Ibis allows you to run the same query against different backends:
- Apache Impala
- Apache Kudu]
- PostgreSQL
- SQLite
- Yandex ClickHouse
- Pandas
- Google BigQuery

("starter code" by https://cloud.google.com/community/tutorials/bigquery-ibis ;)

In [9]:
import ibis
conn = ibis.bigquery.connect(
    project_id='asdf-167312',
    dataset_id='bigquery-public-data.stackoverflow')

In [11]:
table = conn.table('posts_questions')
table

BigQueryTable[table]
  name: bigquery-public-data.stackoverflow.posts_questions
  schema:
    id : int64
    title : string
    body : string
    accepted_answer_id : int64
    answer_count : int64
    comment_count : int64
    community_owned_date : timestamp
    creation_date : timestamp
    favorite_count : int64
    last_activity_date : timestamp
    last_edit_date : timestamp
    last_editor_display_name : string
    last_editor_user_id : int64
    owner_display_name : string
    owner_user_id : int64
    parent_id : string
    post_type_id : int64
    score : int64
    tags : string
    view_count : int64

In [31]:
def percentage_answered(proj):
    has_answer_int = (proj.answer_count > 0).ifelse(1, 0)
    return has_answer_int.mean() * 100

expression = (
    table['creation_date', 'answer_count']
    .mutate(year=lambda x: x.creation_date.year())
    .groupby('year')
    .aggregate(
        total_questions=lambda x: x.count(),
        percentage_answered=percentage_answered,
    )
    .sort_by(ibis.desc(lambda x: x.year))
)

expression.execute()

,year,percentage_answered,total_questions
0,2019,66.506422,1440718
1,2018,77.212207,2008571
2,2017,83.040260,2132515
3,2016,84.753614,2215666
4,2015,86.500011,2211624
5,2014,88.606067,2157464
6,2013,91.688398,2054622
7,2012,94.583315,1641742
8,2011,97.189186,1197767
9,2010,99.044867,692888


## User Defined Functions (UDF)

In [20]:
import ibis.expr.datatypes as dt
from ibis.bigquery import udf

@udf(['double'], 'double')
def example_udf(value):
    return value + 1.0

test_column = ibis.literal(1, type='double')
expression = example_udf(test_column)

print(conn.execute(expression))

2.0


## Joins

Joining literacy rate with country details

In [32]:
edu_table = conn.table(
    'international_education',
    database='bigquery-public-data.world_bank_intl_education')
edu_table = edu_table['value', 'year', 'country_code', 'indicator_code']

country_table = conn.table(
    'country_code_iso',
    database='bigquery-public-data.utility_us')
country_table = country_table['country_name', 'alpha_3_code']

expression = edu_table.join(
    country_table,
    [edu_table.country_code == country_table.alpha_3_code])

conn.execute(
    expression[edu_table.year == 2016]
        # Adult literacy rate.
        [edu_table.indicator_code == 'SE.ADT.LITR.ZS']
        .sort_by([ibis.desc(edu_table.value)])
        .limit(20)
)

,value,year,country_code,indicator_code,country_name,alpha_3_code
0,100.00000,2016,AND,SE.ADT.LITR.ZS,Andorra,AND
1,99.79006,2016,AZE,SE.ADT.LITR.ZS,Azerbaijan,AZE
2,98.84151,2016,SRB,SE.ADT.LITR.ZS,Serbia,SRB
3,98.25051,2016,ESP,SE.ADT.LITR.ZS,Spain,ESP
4,97.12709,2016,VEN,SE.ADT.LITR.ZS,Venezuela,VEN
5,97.04959,2016,SGP,SE.ADT.LITR.ZS,Singapore,SGP
6,96.93023,2016,PSE,SE.ADT.LITR.ZS,West Bank,PSE
7,96.93023,2016,PSE,SE.ADT.LITR.ZS,Gaza Strip,PSE
8,96.53849,2016,MAC,SE.ADT.LITR.ZS,Macau,MAC
9,95.37697,2016,IDN,SE.ADT.LITR.ZS,Indonesia,IDN


## Uploading our own data

- json is not a good choice, especially not if gzip'ed
- very hard to upload non-compressed data to Google Cloud Storage *argh*
- twitter sample data is 19GB/ day uncompressed, 2GB/ day compressed

### Auto-Derive nested schema
```bash
pip install bigquery-schema-generator

zcat twitter-2019-11-24.jl.gz | \
    generate-schema > twitter.schema.json
```

### Upload data to Google Cloud Storage
```bash
gsutil -h "Content-Type: text/plain" -h "Content-Encoding: gzip" \
    -m -o GSUtil:parallel_composite_upload_threshold=150M \
    cp twitter-2019*.gz gs://devfest_twitter/
```

### Load data into BigQuery table
```bash
bq load --source_format NEWLINE_DELIMITED_JSON \
    --ignore_unknown_values \
    --schema twitter.schema.json \
    twitter.tweets \
    gs://devfest_twitter/sample-2019-11-24.jl.gz
```

In [47]:
%%bigquery hashtags --use_legacy_sql
SELECT count(*) as num_ht,
    count(DISTINCT user.screen_name) as num_users,
    HOUR(TIMESTAMP(timestamp_ms / 1000)) as hr, entities.hashtags.text as ht
FROM [asdf-167312.twitter.tweets]
WHERE entities.hashtags.text IS NOT  null and lang == 'en'
GROUP BY hr, ht
HAVING num_ht > 50
ORDER BY hr ASC, num_ht DESC;

In [48]:
import pandas as pd
pd.set_option('display.max_rows', 500)

hashtags

,num_ht,num_users,hr,ht
0,92,90,2,NXTTakeOver
1,81,66,2,BTS
2,69,67,2,WarGames
3,123,114,3,BTS
4,105,105,3,MAMAVOTE
5,68,63,3,GOATBlackFriday
6,67,63,3,WonhoIsOurStrength
7,60,45,3,TwitterEmperorsTHALAFans
8,55,54,3,GOT7
9,55,51,3,나의_모든걸_깨우는_힘
